In [5]:
import cv2
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
%pip install ultralytics
%pip install roboflow
from ultralytics import YOLO

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:

from roboflow import Roboflow
rf = Roboflow(api_key="kOUZOWcGtfa8yegh0ypQ")
project = rf.workspace("thesis-ifxof").project("recyclable-waste")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.17, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Recyclable-Waste--1 in yolov8:: 100%|██████████| 12050/12050 [00:00<00:00, 21367.81it/s]


In [7]:
def resize_image(img, size=(224, 224)):
    return cv2.resize(img, size)


In [9]:
def normalize_image(img):
    return img / 255.0


In [12]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def augment_image(img):
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    aug_iter = datagen.flow(img)
    aug_img = next(aug_iter)[0].astype('uint8')
    return aug_img


In [13]:
def preprocess_image(img_path, save_path):
    # Load the image
    img = cv2.imread(img_path)

    # Resize the image
    #img = resize_image(img)

    # Normalize the image
    img = normalize_image(img)

    # Augment the image
    #img = augment_image(img)
    plt.imsave(save_path, img)


    return img




In [14]:
def ottieni_cartelle(path):
    # Verifica se il percorso fornito esiste ed è una directory
    if not os.path.isdir(path):
        raise ValueError(f"Il percorso {path} non è una directory valida.")

    # Ottieni tutte le cartelle all'interno della directory specificata
    cartelle = [nome for nome in os.listdir(path) if os.path.isdir(os.path.join(path, nome))]

    return cartelle

# Esempio di utilizzo
percorso = 'Recyclable-Waste--1'
cartelle = ottieni_cartelle(percorso)

print("Le cartelle all'interno della directory sono:")
for cartella in cartelle:
    print(cartella)


Le cartelle all'interno della directory sono:
valid
test
train


In [15]:

def preprocess_dataset(input_dir, output_dir):
    for subset in ['train', 'valid', 'test']:
        image_dir = os.path.join(input_dir, subset, 'images')
        label_dir = os.path.join(input_dir, subset, 'labels')

        output_image_dir = os.path.join(output_dir, subset, 'images')
        output_label_dir = os.path.join(output_dir, subset, 'labels')

        os.makedirs(output_image_dir, exist_ok=True)
        os.makedirs(output_label_dir, exist_ok=True)

        for img_file in os.listdir(image_dir):
            img_path = os.path.join(image_dir, img_file)
            save_path = os.path.join(output_image_dir, img_file)
            preprocess_image(img_path, save_path)

            # Copia le annotazioni senza modifiche
            label_file = img_file.replace('.jpg', '.txt')
            label_src_path = os.path.join(label_dir, label_file)
            label_dst_path = os.path.join(output_label_dir, label_file)
            if os.path.exists(label_src_path):
                with open(label_src_path, 'r') as f:
                    annotations = f.readlines()
                with open(label_dst_path, 'w') as f:
                    f.writelines(annotations)

# Esempio di utilizzo
input_dir = 'Recyclable-Waste--1'
output_dir = 'Preprocessed-Recyclable-Waste--1'
preprocess_dataset(input_dir, output_dir)

In [17]:
# Create training config file
import yaml
import os
#os.rename("GARBAGE-DETECTION-3","datasets")
dataset = 'Preprocessed-Recyclable-Waste--1'
labels_name = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
data = {
    "train": f"{dataset}/train",
    "val":   f"{dataset}/valid",
    "test":  f"{dataset}/test",
    "nc": len(labels_name),
    "names": labels_name,
}
# Create and write to the YAML file
yaml_file_path = f"{dataset}/data.yaml"
with open(yaml_file_path, 'w') as file:
    yaml.dump(data, file, sort_keys=False)

In [18]:
model = YOLO('yolov9c.pt', task='detect')

100%|██████████| 49.4M/49.4M [00:04<00:00, 12.7MB/s]


In [20]:
torch.cuda.is_available()

True

In [19]:
torch.cuda.empty_cache()
t_dir = '/home/chiavetta/Scrivania/training_embedded/Preprocessed-Recyclable-Waste--1/data.yaml'
model.train(data=t_dir, batch=-1, epochs=15, project='tmp')

New https://pypi.org/project/ultralytics/8.2.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.17 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7940MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/home/chiavetta/Scrivania/training_embedded/Preprocessed-Recyclable-Waste--1/data.yaml, epochs=15, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=tmp, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, a

RuntimeError: Dataset '/home/chiavetta/Scrivania/training_embedded/Preprocessed-Recyclable-Waste--1/data.yaml' error ❌ 
Dataset '/home/chiavetta/Scrivania/training_embedded/Preprocessed-Recyclable-Waste--1/data.yaml' images not found ⚠️, missing path '/home/chiavetta/Scrivania/training_embedded/Preprocessed-Recyclable-Waste--1/Preprocessed-Recyclable-Waste--1/valid'
Note dataset download directory is '/home/chiavetta/Scrivania/training_for_trash/datasets'. You can update this in '/home/chiavetta/.config/Ultralytics/settings.yaml'